In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import keras.backend as K
import tensorflow as tf
import warnings

# 경고 비활성화
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ASAC_4기_기업프로젝트_SK하이닉스/1. 데이터 전처리/5. 최종 데이터셋(2024.00.00 기준)/final_df_reordered.csv')

In [ ]:
df=df[[
 'RCP_ID',
 'STEP_NM',
 'TA_tm_before',
 'start_dt_tm',
 'end_dt_tm',
 'TA_tm_after',
 'P_Time',
 'STEP_2',
 'STEP_8',
 'STEP_9',
 'STEP_11',
 'STEP_17',
 'STEP_19',
 'STEP_20',
 'STEP_21',
 'STEP_24',
 'STEP_25',
 'STEP_26',
 'STEP_27',
 'STEP_28',
 'STEP_34',
 'STEP_36',
 'STEP_38',
 'last_hours',
 'sequence',
 'max_val_1',
 'max_val_2',
 'max_val_3',
 'max_val_4',
 'max_val_5',
 'max_val_6',
 'max_val_7',
 'max_val_8',
 'max_val_9',
 'max_val_10',
 'max_val_11',
 'max_val_12',
 'max_val_13',
 'max_val_14',
 'max_val_15',
 'max_val_16',
 'max_val_17',
 'max_val_18',
 'max_val_19',
 'mean_val_1',
 'mean_val_2',
 'mean_val_3',
 'mean_val_4',
 'mean_val_5',
 'mean_val_6',
 'mean_val_7',
 'mean_val_8',
 'mean_val_9',
 'mean_val_10',
 'mean_val_11',
 'mean_val_12',
 'mean_val_13',
 'mean_val_14',
 'mean_val_15',
 'mean_val_16',
 'mean_val_17',
 'mean_val_18',
 'mean_val_19',
 'median_val_1',
 'median_val_2',
 'median_val_3',
 'median_val_4',
 'median_val_5',
 'median_val_6',
 'median_val_7',
 'median_val_8',
 'median_val_9',
 'median_val_10',
 'median_val_11',
 'median_val_12',
 'median_val_13',
 'median_val_14',
 'median_val_15',
 'median_val_16',
 'median_val_17',
 'median_val_18',
 'median_val_19',
 'min_val_1',
 'min_val_2',
 'min_val_3',
 'min_val_4',
 'min_val_5',
 'min_val_6',
 'min_val_7',
 'min_val_8',
 'min_val_9',
 'min_val_10',
 'min_val_11',
 'min_val_12',
 'min_val_13',
 'min_val_14',
 'min_val_15',
 'min_val_16',
 'min_val_17',
 'min_val_18',
 'min_val_19',
 'range_val_1',
 'range_val_2',
 'range_val_3',
 'range_val_4',
 'range_val_5',
 'range_val_6',
 'range_val_7',
 'range_val_8',
 'range_val_9',
 'range_val_10',
 'range_val_11',
 'range_val_12',
 'range_val_13',
 'range_val_14',
 'range_val_15',
 'range_val_16',
 'range_val_17',
 'range_val_18',
 'range_val_19',
 'stddev_val_1',
 'stddev_val_2',
 'stddev_val_3',
 'stddev_val_4',
 'stddev_val_5',
 'stddev_val_6',
 'stddev_val_7',
 'stddev_val_8',
 'stddev_val_9',
 'stddev_val_10',
 'stddev_val_11',
 'stddev_val_12',
 'stddev_val_13',
 'stddev_val_14',
 'stddev_val_15',
 'stddev_val_16',
 'stddev_val_17',
 'stddev_val_18',
 'stddev_val_19',
 'diff_hours']]

In [ ]:
numeric_columns =list(df.columns[23:-1])

In [ ]:
train_split = 0
for i in df['TA_tm_before'].unique():
    train_split +=len(df[df['TA_tm_before'] == i])
    if train_split > 38500:
        break

In [ ]:
X,Y = df.iloc[:, :-1], df.iloc[:, -1]

In [ ]:
X_train,X_test= X[:train_split],X[train_split:]
Y_train,Y_test= Y[:train_split],Y[train_split:]

In [ ]:
for i in X_train.columns:
    if i in numeric_columns:
        scaler_xtr = StandardScaler()
        scaler_xtr.fit(X_train[[i]])
        X_train[i] = scaler_xtr.transform(X_train[[i]])

for i in X_test.columns:
    if i in numeric_columns:
        scaler_xts = StandardScaler()
        scaler_xts.fit(X_test[[i]])
        X_test[i] = scaler_xts.transform(X_test[[i]])


In [ ]:
Y_train = pd.DataFrame(Y_train)
Y_test = pd.DataFrame(Y_test)
Y_train.reset_index(drop=True, inplace=True)
Y_test.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
X_train.reset_index(drop = True, inplace = True)

# METRIC

In [ ]:
def weighted_rmse1(y_true, y_pred):
    diff = tf.square(y_true - y_pred)
    rul_t = 1 - (y_true / tf.reduce_max(y_true))
    n_metric = diff * rul_t
    return (tf.reduce_sum(n_metric) / tf.reduce_sum(rul_t)) ** 0.5

def weighted_rmse2(y_true, y_pred):
    diff = tf.square(y_true - y_pred)
    rul_t = 1 / (y_true+1)
    weighted_diff = diff * rul_t
    weighted_rmse = tf.sqrt(tf.reduce_sum(weighted_diff) / tf.reduce_sum(rul_t))
    return weighted_rmse

# 이거 사용

In [ ]:
def custom_metric(y_true, y_pred):
    diff = tf.abs(y_true - y_pred)
    rul_t = 1 - (y_true / tf.reduce_max(y_true))
    n_metric = diff * rul_t
    return tf.reduce_mean(n_metric)

np.random.seed(0)
tf.random.set_seed(0)
look_back = 50
model = Sequential()
model.add(LSTM(128, input_shape=(look_back,132), return_sequences=True))
model.add(LSTM(64,return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
# model.compile(optimizer='adam', loss='mae', metrics=['mae', custom_metric])
model.compile(loss='mean_squared_error', optimizer='adam', metrics=[custom_metric])

from sklearn.metrics import mean_absolute_error

for x in X_train['TA_tm_before'].unique():
    temp_df = X_train[X_train['TA_tm_before'] == x]
    temp_df_t = Y_train.loc[temp_df.index]

    x_trains, y_trains = [], []
    for i in range(len(temp_df)-look_back):
        x_Train = temp_df.iloc[i:i+look_back,:]
        y_Train = temp_df_t.iloc[i+look_back]
        x_trains.append(x_Train.drop(columns=['RCP_ID','STEP_NM','TA_tm_before','start_dt_tm','end_dt_tm','TA_tm_after','P_Time']))
        y_trains.append(y_Train)

    x_train, y_train = np.array(x_trains), np.array(y_trains)

    if len(x_train) > 300:
        model.fit(x_train, y_train, epochs=100, batch_size=50, verbose=1)

        # 예측
        y_pred = model.predict(x_train)

        # 평가
        mae = mean_absolute_error(y_train, y_pred)
        print(f"MAE for TA_tm_before={x}: {mae}")
        custom_score = custom_metric(y_train, y_pred)
        print(f"Custom metric for TA_tm_before={x}: {custom_score}")

from sklearn.metrics import mean_absolute_error

for x in X_test['TA_tm_before'].unique():
    x_tests, y_tests = [], []
    temp_df = X_test[X_test['TA_tm_before'] == x]
    temp_df_t = Y_test.loc[temp_df.index]

    if len(temp_df) > look_back:
        for i in range(len(temp_df)-look_back):
            x_Test = temp_df.iloc[i:i+look_back,:]
            y_Test = temp_df_t.iloc[i+look_back]
            x_tests.append(x_Test.drop(columns=['RCP_ID','STEP_NM','TA_tm_before','start_dt_tm','end_dt_tm','TA_tm_after','P_Time']))
            y_tests.append(y_Test)

        x_test, y_test = np.array(x_tests), np.array(y_tests)
        testPredict = model.predict(x_test).reshape(-1)

        # custom_metric을 이용한 평가
        custom_score = custom_metric(y_test, testPredict)
        print(f"Custom metric for TA_tm_before={x}: {custom_score}")

        # 평가
        mae = mean_absolute_error(y_test, testPredict)
        print(f"MAE for TA_tm_before={x}: {mae}")

        # 그래프
        temp_df['start_dt_tm'] = pd.to_datetime(temp_df['start_dt_tm'])
        temp_df.sort_values('start_dt_tm', inplace=True)

        plt.figure(figsize=(15, 5))
        plt.plot(temp_df['start_dt_tm'][50:], testPredict, marker='o', linestyle='-', label='Predictions')
        plt.plot(temp_df['start_dt_tm'][50:], temp_df_t[50:], marker='o', linestyle='-', label='True values')
        plt.title('Diff Hours Over Time')
        plt.xlabel('Start Date Time')
        plt.ylabel('Diff Hours')
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.show()



Epoch 1/100
130/130 [==============================] - 6s 11ms/step - loss: 7294.6387 - custom_metric: 22.1053
Epoch 2/100
130/130 [==============================] - 1s 10ms/step - loss: 6519.5830 - custom_metric: 20.0174
Epoch 3/100
130/130 [==============================] - 1s 10ms/step - loss: 5971.3022 - custom_metric: 18.6719
Epoch 4/100
130/130 [==============================] - 2s 13ms/step - loss: 5484.7363 - custom_metric: 17.7177
Epoch 5/100
130/130 [==============================] - 2s 14ms/step - loss: 5047.4365 - custom_metric: 16.7243
Epoch 6/100
130/130 [==============================] - 1s 11ms/step - loss: 4654.6133 - custom_metric: 16.1149
Epoch 7/100
130/130 [==============================] - 1s 10ms/step - loss: 4302.9087 - custom_metric: 15.6332
Epoch 8/100
130/130 [==============================] - 1s 10ms/step - loss: 3989.0352 - custom_metric: 15.3188
Epoch 9/100
130/130 [==============================] - 1s 10ms/step - loss: 3708.6926 - custom_metric: 15.2526
E